# Plan
Ok, so my plan is basically just to break the dataset specifically the date time into different catagorical values. Specifically two new coluns, first the year, a numerical value in order to account for the possibility of a raising or lowering of avg temperachure over a few years, and a second a value 1-356 that gives which day of the specified year it is. I think that by doing it this way the model will be able to diferentiate between the different times of the year without having to deal with wierd time series and all that crazy stuff. Basically I just need to find a model that will train off of that data based on the individual values.

Tuturial I am going to use:https://machinelearninggeek.com/multi-layer-perceptron-neural-network-using-python/

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/wids-all-datasets/train_data.csv
/kaggle/input/wids-all-datasets/test_data.csv
/kaggle/input/wids-all-datasets/sample_solution.csv


# Functions
The functions that I will be using in the process of training the model

In [2]:
#Function to convert dataframe to prefered format.
def df_format_neumerical(df):
    df['dayofyear']=0
    df['year']=0
    i=0
    for i in df.index.values.tolist():
        i=int(i)
        period=pd.Period(freq='D')
        df['dayofyear'][i]=int(period.day_of_year)
        df['year'][i]=int(period.year)
        i+=1
    df.pop('startdate')
    neumerical_df=df[:]
    neumerical_df.pop('climateregions__climateregion')
    return df,neumerical_df

In [3]:
#Sepperate dataframe into test and train
def seperate_train(df):
    seventy_cutoff=int(.7*len(df))
    ninty_cutoff=int(len(df)-(.1*len(df)))
    train_neumerical_df=neumerical_df[:seventy_cutoff]
    test_neumerical_df=neumerical_df[ninty_cutoff:len(df)]
    return train_neumerical_df, test_neumerical_df

In [4]:
def df_seperate_y(df):
    y_df=df['contest-tmp2m-14d__tmp2m']
    X_df=df.drop('contest-tmp2m-14d__tmp2m',axis=1)
    return y_df,X_df

In [5]:
train_df=pd.read_csv('/kaggle/input/wids-all-datasets/train_data.csv')
train_df=train_df.dropna(axis=0)
test_df=pd.read_csv('/kaggle/input/wids-all-datasets/test_data.csv')
test_df=test_df.dropna(axis=0)

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 352604 entries, 0 to 375733
Columns: 246 entries, index to wind-vwnd-925-2010-20
dtypes: float64(240), int64(4), object(2)
memory usage: 664.5+ MB


In [7]:
#train_df,train_neumerical_df=df_format_neumerical(train_df)
#test_df, test_neumerical_df=df_format_neumerical(test_df)

In [8]:
train_neumerical_df=train_df
test_neumerical_df=test_df

In [9]:
train_neumerical_df=train_neumerical_df.drop(['startdate','climateregions__climateregion'], axis=1)
test_neumerical_df=test_neumerical_df.drop(['startdate','climateregions__climateregion'],axis=1)


Ok, so I think that with that I am ready to actually start on the model that I am trying to train.

In [10]:
#df=pd.read_csv('/kaggle/input/wids-all-datasets/train_data.csv')
#df.dropna()

In [11]:
# Import LabelEncoder
from sklearn import preprocessing

# Creating labelEncoder
le = preprocessing.LabelEncoder()

# Converting string labels into numbers.
train_df['climateregions__climateregion']=le.fit_transform(train_df['climateregions__climateregion'])
test_df['climateregions__climateregion']=le.fit_transform(test_df['climateregions__climateregion'])

In [12]:
y_train_neumerical_df,X_train_neumerical_df=df_seperate_y(train_neumerical_df)

In [13]:
y_train_neumerical_df=y_train_neumerical_df.astype('int')

In [14]:
# Import MLPClassifer 
from sklearn.neural_network import MLPClassifier

# Create model object
clf = MLPClassifier(hidden_layer_sizes=(2400,2000),
                    random_state=100,
                    verbose=True,
                    learning_rate_init=0.01)

# Fit data onto the model
clf.fit(X_train_neumerical_df,y_train_neumerical_df)

Iteration 1, loss = 4.16668728
Iteration 2, loss = 3.67217139
Iteration 3, loss = 3.67199293
Iteration 4, loss = 3.67174124
Iteration 5, loss = 3.67145683
Iteration 6, loss = 3.67108475
Iteration 7, loss = 3.67070933
Iteration 8, loss = 3.68666733
Iteration 9, loss = 3.69620857
Iteration 10, loss = 3.72155545
Iteration 11, loss = 3.66907586
Iteration 12, loss = 3.81971894
Iteration 13, loss = 4.19255440
Iteration 14, loss = 3.55345499
Iteration 15, loss = 3.66173617
Iteration 16, loss = 3.77667385
Iteration 17, loss = 3.67044564
Iteration 18, loss = 3.67036133
Iteration 19, loss = 3.67000650
Iteration 20, loss = 3.66987696
Iteration 21, loss = 3.66959355
Iteration 22, loss = 3.66941406
Iteration 23, loss = 3.66907086
Iteration 24, loss = 3.66868492
Iteration 25, loss = 3.66820459
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


MLPClassifier(hidden_layer_sizes=(2400, 2000), learning_rate_init=0.01,
              random_state=100, verbose=True)

In [15]:
X_train_neumerical_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 352604 entries, 0 to 375733
Columns: 243 entries, index to wind-vwnd-925-2010-20
dtypes: float64(239), int64(4)
memory usage: 656.4 MB


In [16]:
test_neumerical_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31354 entries, 0 to 31353
Columns: 243 entries, index to wind-vwnd-925-2010-20
dtypes: float64(241), int64(2)
memory usage: 58.4 MB


In [17]:
ypred=clf.predict(test_neumerical_df)

In [18]:
ypred

array([0, 0, 0, ..., 0, 0, 0])

In [19]:
#Trying to create a final 
final=pd.DataFrame()
final['contest-tmp2m-14d__tmp2m']=ypred
final['# index']=0
i=len(train_df) #Legnth of train_data.csv
while i<len(train_df):
    final['# index'][i-len(train_df)]=i
    i+=1
final

#I basically need to count up from len(train_data.csv+1)
#to len(test_data.csv)+len(train_data.csv)

,contest-tmp2m-14d__tmp2m,# index
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
31349,0,0
31350,0,0
31351,0,0
31352,0,0


In [20]:
final.to_csv('finalV1.csv',index=False)

In [21]:
final

,contest-tmp2m-14d__tmp2m,# index
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
31349,0,0
31350,0,0
31351,0,0
31352,0,0
